<a href="https://en.wikipedia.org/wiki/The_Accidental_Tourist_(film)">
<img src = "https://images-na.ssl-images-amazon.com/images/I/91hXwldu4lL._RI_.jpg" width = 400> </a>

<h1 align=center><font size = 5>The Accidental Tourist Reccomendation System</font></h1>

<h1 align=center><font size = 3>This notebook was made for the travelers that live in NY and want to visit Toronto. Even far from their neighborhoods, they want to have the same conveniences they have in the neighborhood they live. They don't want to be badly surprised with having no venues around that they are not used to. So the user will pick the neighborhood he lives in NY from a list, and then the algorithm will find the similar neighborhoods in Toronto. </font></h1>

First, we import all the libraries we need:

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         393 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0

The following packages will be UPDATED:

  ca-certificates                     2020.4.5

In [2]:
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension
import ipywidgets as widgets

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


Then we start importing the data from Toronto:

In [3]:
## I've copied the table of neighborhoods from Wikipedia, and saved it to a CSV file.
## Let's put it to a Pandas dataframe:
wiki_toronto_neighborhoods_raw = pd.read_csv('Wiki_Toronto_Neighborhoods.csv')
print('Shape of Dataframe: ',wiki_toronto_neighborhoods_raw.shape)

## Rename column so it gets exactly like instructed by Alex:
wiki_toronto_neighborhoods_raw.rename(columns={"Postal code": "Postalcode"}, inplace=True)
wiki_toronto_neighborhoods_raw.head()

Shape of Dataframe:  (183, 3)


,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [4]:
## Remove the 'Not assigned' Boroughs
condition = wiki_toronto_neighborhoods_raw['Borough'] != 'Not assigned'
Toronto_Neighb = wiki_toronto_neighborhoods_raw[condition]
print('Shape of Dataframe after removing "Not assigned" Boroughs: ',Toronto_Neighb.shape)

## Also, remove the records with Nan in any columns, to clean the table (there were only 3)
Toronto_Neighb.dropna(how='any', inplace = True)
Toronto_Neighb.reset_index(drop=True, inplace=True)
print('Shape of Dataframe after removing NaN values: ',Toronto_Neighb.shape)
Toronto_Neighb.head()

## There is no need to aggregate records (as asked by Alex) since the Wikipedia page is updated
## So this line keeps commentend:
## Toronto_Neighb = Toronto_Neighb.groupby(level=['Postalcode'], sort=False).agg( ','.join)

Shape of Dataframe after removing "Not assigned" Boroughs:  (106, 3)
Shape of Dataframe after removing NaN values:  (103, 3)


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [5]:
Toronto_Neighb.sort_values('Neighborhood')

,Postalcode,Borough,Neighborhood
78,M1S,Scarborough,Agincourt
93,M8W,Etobicoke,Alderwood / Long Branch
28,M3H,North York,Bathurst Manor / Wilson Heights / Downsview North
39,M2K,North York,Bayview Village
55,M5M,North York,Bedford Park / Lawrence Manor East
20,M5E,Downtown Toronto,Berczy Park
58,M1N,Scarborough,Birch Cliff / Cliffside West
43,M6K,West Toronto,Brockton / Parkdale Village / Exhibition Place
100,M7Y,East Toronto,Business reply mail Processing CentrE
87,M5V,Downtown Toronto,CN Tower / King and Spadina / Railway Lands / ...


In [6]:
## Now let's get the coordinates from other table. I've decided to get directly from the CSV file
## So let's put it in this dataframe:

coord = pd.read_csv('Geospatial_Coordinates.csv')
coord.rename(columns={"Postal Code": "Postalcode"}, inplace=True)
coord.head()

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
## Now let's create two new columns to accomodate the geolocation data:
Toronto_Neighb['Latitude'] = np.nan
Toronto_Neighb['Longitude'] = np.nan
Toronto_Neighb.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,NaN,NaN
1,M4A,North York,Victoria Village,NaN,NaN
2,M5A,Downtown Toronto,Regent Park / Harbourfront,NaN,NaN
3,M6A,North York,Lawrence Manor / Lawrence Heights,NaN,NaN
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,NaN,NaN


In [8]:
## We want to update the values of "coord" into "Toronto_Neighb"
## I will use the update method, so first I need to equalize the indexes of both dataframes
Toronto_Neighb.set_index('Postalcode', inplace=True)
coord.set_index('Postalcode', inplace=True)

In [9]:
## As you can see, now the Postalcode is the index:
Toronto_Neighb.head()

,Borough,Neighborhood,Latitude,Longitude
Postalcode,,,,
M3A,North York,Parkwoods,NaN,NaN
M4A,North York,Victoria Village,NaN,NaN
M5A,Downtown Toronto,Regent Park / Harbourfront,NaN,NaN
M6A,North York,Lawrence Manor / Lawrence Heights,NaN,NaN
M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,NaN,NaN


In [10]:
## ...as well as in the coord dataframe:
coord.head()

,Latitude,Longitude
Postalcode,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [11]:
## Now we can use the update method, which update the columns of same name, based on the index:
Toronto_Neighb.update(coord)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:5732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = expressions.where(mask, this, that)


In [12]:
## Now let's check if Latitude and Longitude are completed:
Toronto_Neighb.head()

,Borough,Neighborhood,Latitude,Longitude
Postalcode,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


In [13]:
## Let's reset the index so our main dataframe gets back to what it was:
Toronto_Neighb.reset_index(inplace=True)

In [14]:
Toronto_Neighb.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


All set for Toronto! Now let's work on NY data.

Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood. 

Luckily, this dataset exists for free on the web. Feel free to try to find this dataset on your own, but here is the link to the dataset: https://geo.nyu.edu/catalog/nyu_2451_34572

For your convenience, I downloaded the files and placed it on the server, so you can simply run a `wget` command and access the data. So let's go ahead and do that.

In [15]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [16]:
#Next, let's load the data.
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [17]:
#Notice how all the relevant data is in the *features* key,
#which is basically a list of the neighborhoods.
#So, let's define a new variable that includes this data.
neighborhoods_data = newyork_data['features']

In [18]:
#Let's take a look at the first item in this list.
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [19]:
# The next task is essentially transforming this data of nested
#  Python dictionaries into a *pandas* dataframe.
# So let's start by creating an empty dataframe.

In [20]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods_NY = pd.DataFrame(columns=column_names)

In [21]:
#Take a look at the empty dataframe to confirm that the columns are as intended.
neighborhoods_NY

,Borough,Neighborhood,Latitude,Longitude


Then let's loop through the data and fill the dataframe one row at a time.

In [22]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods_NY = neighborhoods_NY.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [23]:
#Quickly examine the resulting dataframe.
neighborhoods_NY.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


And make sure that the dataset has all 5 boroughs and 306 neighborhoods.

In [24]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods_NY['Borough'].unique()),
        neighborhoods_NY.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [25]:
# We will merge both dataframes, NY and TO, so let's include the city on them
neighborhoods_NY["city"] = "NY"
neighborhoods_NY["Postalcode"] = "NY"
neighborhoods_NY.head()

,Borough,Neighborhood,Latitude,Longitude,city,Postalcode
0,Bronx,Wakefield,40.894705,-73.847201,NY,NY
1,Bronx,Co-op City,40.874294,-73.829939,NY,NY
2,Bronx,Eastchester,40.887556,-73.827806,NY,NY
3,Bronx,Fieldston,40.895437,-73.905643,NY,NY
4,Bronx,Riverdale,40.890834,-73.912585,NY,NY


In [26]:
Toronto_Neighb["city"] = "TO"
Toronto_Neighb.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Postalcode,Borough,Neighborhood,Latitude,Longitude,city
0,M3A,North York,Parkwoods,43.753259,-79.329656,TO
1,M4A,North York,Victoria Village,43.725882,-79.315572,TO
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,TO
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763,TO
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494,TO


In [27]:
# Finally, we concatenate to one single Dataframe
neighb_NY_TO = pd.concat([neighborhoods_NY,Toronto_Neighb])
neighb_NY_TO.head()

,Borough,Neighborhood,Latitude,Longitude,city,Postalcode
0,Bronx,Wakefield,40.894705,-73.847201,NY,NY
1,Bronx,Co-op City,40.874294,-73.829939,NY,NY
2,Bronx,Eastchester,40.887556,-73.827806,NY,NY
3,Bronx,Fieldston,40.895437,-73.905643,NY,NY
4,Bronx,Riverdale,40.890834,-73.912585,NY,NY


## Now here the user process starts!

In [28]:
# Creating a Picklist with NY Neighborhoods:
dropdown_Neighb = widgets.Dropdown(options = neighborhoods_NY.Neighborhood.sort_values(ascending=True),
                                value = "Allerton",description='Neighborhood:',disabled=False,)

In [29]:
from ipywidgets import interact, interactive, fixed, interact_manual

caption = widgets.Label(value=' ')

caption.value = ' '
def dropdown_Neighb_eventhandler(change):
    caption.value = change.new


In [30]:
dropdown_Neighb.observe(dropdown_Neighb_eventhandler, names='value') 

## Select your NY neighborhood on the list below, so later the system will find similar Neighborhoods in Toronto:

In [31]:
display(dropdown_Neighb)

Dropdown(description='Neighborhood:', options=('Allerton', 'Annadale', 'Arden Heights', 'Arlington', 'Arrochar…

In [32]:
print("The value selected is: "+caption.value)

The value selected is:  


In [40]:
if caption.value == " ":
    print('No value selected, defaulted to Allerton');
    caption.value = "Allerton"

No value selected, defaulted to Allerton


In [41]:
# Now let's concatenate the Toronto Dataframe with the record selected by the user:
# You will notice the NY record will be the first on the list.

Toronto_Neighb = Toronto_Neighb[(Toronto_Neighb['city']=='TO')]
Toronto_Neighb = pd.concat([neighborhoods_NY[(neighborhoods_NY['Neighborhood']==caption.value)],Toronto_Neighb])
Toronto_Neighb

,Borough,Neighborhood,Latitude,Longitude,city,Postalcode
298,Bronx,Allerton,40.865788,-73.859319,NY,NY
0,North York,Parkwoods,43.753259,-79.329656,TO,M3A
1,North York,Victoria Village,43.725882,-79.315572,TO,M4A
2,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,TO,M5A
3,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763,TO,M6A
4,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494,TO,M7A
5,Etobicoke,Islington Avenue,43.667856,-79.532242,TO,M9A
6,Scarborough,Malvern / Rouge,43.806686,-79.194353,TO,M1B
7,North York,Don Mills,43.745906,-79.352188,TO,M3B
8,East York,Parkview Hill / Woodbine Gardens,43.706397,-79.309937,TO,M4B


In [42]:
# here is the isolated record:
neighborhoods_NY[(neighborhoods_NY['Neighborhood']==caption.value)]

,Borough,Neighborhood,Latitude,Longitude,city,Postalcode
298,Bronx,Allerton,40.865788,-73.859319,NY,NY


In [43]:
# Now let's get the venues on Foursquare!

In [44]:
#### Define Foursquare Credentials and Version

CLIENT_ID = 'VAHUGBJMS1OYZJ5GCHJGF121BRMAOOBZWCD135K3XEJ1ISCG' # your Foursquare ID
CLIENT_SECRET = 'CNQDHEV0R4P5YU1YPOSUAF0F2Y3QHFNLVETSRFZEEFMW0DUE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

## Define other variables to Foursquare API
radius = 500
LIMIT = 100

Your credentails:
CLIENT_ID: VAHUGBJMS1OYZJ5GCHJGF121BRMAOOBZWCD135K3XEJ1ISCG
CLIENT_SECRET:CNQDHEV0R4P5YU1YPOSUAF0F2Y3QHFNLVETSRFZEEFMW0DUE


In [45]:
## Exploring Neighborhoods in Toronto!

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [46]:
#### Running the above function on each neighborhood and
## creating a new dataframe called *toronto_venues*.

toronto_venues = getNearbyVenues(names=Toronto_Neighb['Neighborhood'],
                                   latitudes=Toronto_Neighb['Latitude'],
                                   longitudes=Toronto_Neighb['Longitude'],
                                  )



Allerton
Parkwoods
Victoria Village
Regent Park / Harbourfront
Lawrence Manor / Lawrence Heights
Queen's Park / Ontario Provincial Government
Islington Avenue
Malvern / Rouge
Don Mills
Parkview Hill / Woodbine Gardens
Garden District-Ryerson
Glencairn
West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale
Rouge Hill / Port Union / Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood
Guildwood / Morningside / West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor / Wilson Heights / Downsview North
Thorncliffe Park
Richmond / Adelaide / King
Dufferin / Dovercourt Village
Scarborough Village
Fairview / Henry Farm / Oriole
Northwood Park / York University
East Toronto
Harbourfront East / Union Station / Toronto Islands
Little Portugal / Trinity
Kennedy Park / Ionview / East Birchmount Park
Bayview Vi

In [47]:
#### Let's check the size of the resulting dataframe

print(toronto_venues.shape)
toronto_venues.head()

(2159, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Allerton,40.865788,-73.859319,Domenick's Pizzeria,40.865576,-73.858124,Pizza Place
1,Allerton,40.865788,-73.859319,White Castle,40.866065,-73.862307,Fast Food Restaurant
2,Allerton,40.865788,-73.859319,Sal & Doms Bakery,40.865377,-73.855236,Dessert Shop
3,Allerton,40.865788,-73.859319,Bronx Martial Arts Academy,40.865721,-73.857529,Martial Arts Dojo
4,Allerton,40.865788,-73.859319,Dunkin',40.865204,-73.859007,Donut Shop


In [48]:
# Let's check how many venues were returned for each neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
Alderwood / Long Branch,8,8,8,8,8,8
Allerton,29,29,29,29,29,29
Bathurst Manor / Wilson Heights / Downsview North,20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
Bedford Park / Lawrence Manor East,23,23,23,23,23,23
Berczy Park,56,56,56,56,56,56
Birch Cliff / Cliffside West,5,5,5,5,5,5
Brockton / Parkdale Village / Exhibition Place,24,24,24,24,24,24


In [49]:
## Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 274 uniques categories.


In [50]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Allerton,40.865788,-73.859319,Domenick's Pizzeria,40.865576,-73.858124,Pizza Place
1,Allerton,40.865788,-73.859319,White Castle,40.866065,-73.862307,Fast Food Restaurant
2,Allerton,40.865788,-73.859319,Sal & Doms Bakery,40.865377,-73.855236,Dessert Shop
3,Allerton,40.865788,-73.859319,Bronx Martial Arts Academy,40.865721,-73.857529,Martial Arts Dojo
4,Allerton,40.865788,-73.859319,Dunkin',40.865204,-73.859007,Donut Shop


In [51]:
# Let's get only the venues that appears more than 25 times, so our clustering gets easier
venues_count = toronto_venues.groupby("Venue Category")["Neighborhood"].count().sort_values(ascending=False)
venues_count = venues_count.head(25)
venues_count 

Venue Category
Coffee Shop             174
Café                     98
Restaurant               71
Pizza Place              53
Park                     51
Bakery                   45
Italian Restaurant       45
Sandwich Place           42
Hotel                    41
Japanese Restaurant      39
Clothing Store           34
Gym                      33
Sushi Restaurant         30
Grocery Store            30
Bar                      29
Fast Food Restaurant     27
American Restaurant      26
Pub                      26
Bank                     25
Pharmacy                 23
Breakfast Spot           23
Thai Restaurant          21
Seafood Restaurant       20
Diner                    19
Ice Cream Shop           19
Name: Neighborhood, dtype: int64

In [52]:
toronto_venues = toronto_venues[toronto_venues["Venue Category"].isin(venues_count.index)]
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Allerton,40.865788,-73.859319,Domenick's Pizzeria,40.865576,-73.858124,Pizza Place
1,Allerton,40.865788,-73.859319,White Castle,40.866065,-73.862307,Fast Food Restaurant
5,Allerton,40.865788,-73.859319,IHOP,40.865728,-73.862460,Breakfast Spot
7,Allerton,40.865788,-73.859319,Domino's Pizza,40.866363,-73.861798,Pizza Place
9,Allerton,40.865788,-73.859319,Rite Aid,40.865949,-73.860922,Pharmacy


In [53]:
## Analyze Each Neighborhood

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe 
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
#fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
#toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,American Restaurant,Bakery,Bank,Bar,Breakfast Spot,Café,Clothing Store,Coffee Shop,Diner,Fast Food Restaurant,Grocery Store,Gym,Hotel,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Park,Pharmacy,Pizza Place,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Sushi Restaurant,Thai Restaurant,Neighborhood
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,Allerton
1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Allerton
5,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Allerton
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,Allerton
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,Allerton


In [54]:
## Moving the Neighborhood column to front:

cols = toronto_onehot.columns.tolist()
cols.remove('Neighborhood') ## removing the column from the middle... 
cols.insert(0,'Neighborhood') ## ...and reinserting it on the first position
toronto_onehot = toronto_onehot[cols]
toronto_onehot.head()

,Neighborhood,American Restaurant,Bakery,Bank,Bar,Breakfast Spot,Café,Clothing Store,Coffee Shop,Diner,Fast Food Restaurant,Grocery Store,Gym,Hotel,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Park,Pharmacy,Pizza Place,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Sushi Restaurant,Thai Restaurant
0,Allerton,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,Allerton,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Allerton,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,Allerton,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
9,Allerton,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [55]:
# And let's examine the new dataframe size.
toronto_onehot.shape

(1044, 26)

In [56]:
# Next, let's group rows by neighborhood and by taking the mean of the frequency
#  of occurrence of each category

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,American Restaurant,Bakery,Bank,Bar,Breakfast Spot,Café,Clothing Store,Coffee Shop,Diner,Fast Food Restaurant,Grocery Store,Gym,Hotel,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Park,Pharmacy,Pizza Place,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Sushi Restaurant,Thai Restaurant
0,Agincourt,0.0,0.0,0.000000,0.0,0.500,0.000000,0.5,0.000000,0.000000,0.000,0.000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
1,Alderwood / Long Branch,0.0,0.0,0.000000,0.0,0.000,0.000000,0.0,0.166667,0.000000,0.000,0.000,0.166667,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.333333,0.166667,0.000000,0.166667,0.0,0.000000,0.0
2,Allerton,0.0,0.0,0.000000,0.0,0.125,0.000000,0.0,0.000000,0.000000,0.125,0.125,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.125000,0.500000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
3,Bathurst Manor / Wilson Heights / Downsview North,0.0,0.0,0.181818,0.0,0.000,0.000000,0.0,0.181818,0.090909,0.000,0.000,0.000000,0.0,0.090909,0.0,0.000000,0.0,0.090909,0.090909,0.000000,0.090909,0.090909,0.0,0.090909,0.0
4,Bayview Village,0.0,0.0,0.333333,0.0,0.000,0.333333,0.0,0.000000,0.000000,0.000,0.000,0.000000,0.0,0.000000,0.0,0.333333,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0


In [57]:
#### Let's confirm the new size
toronto_grouped.shape

(86, 26)

In [58]:
#### Let's print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                 venue  freq
0       Breakfast Spot   0.5
1       Clothing Store   0.5
2  American Restaurant   0.0
3   Italian Restaurant   0.0
4     Sushi Restaurant   0.0


----Alderwood / Long Branch----
            venue  freq
0     Pizza Place  0.33
1  Sandwich Place  0.17
2             Pub  0.17
3     Coffee Shop  0.17
4             Gym  0.17


----Allerton----
                  venue  freq
0           Pizza Place  0.50
1        Breakfast Spot  0.12
2  Fast Food Restaurant  0.12
3         Grocery Store  0.12
4              Pharmacy  0.12


----Bathurst Manor / Wilson Heights / Downsview North----
              venue  freq
0              Bank  0.18
1       Coffee Shop  0.18
2  Sushi Restaurant  0.09
3    Sandwich Place  0.09
4        Restaurant  0.09


----Bayview Village----
                 venue  freq
0                 Bank  0.33
1                 Café  0.33
2  Japanese Restaurant  0.33
3  American Restaurant  0.00
4   Italian Restaurant  0.00


----Bedford P

In [59]:
# Let's put that into a *pandas* dataframe
# First, let's write a function to sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [60]:
# Now let's create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Clothing Store,Thai Restaurant,Gym,Bakery,Bank,Bar,Café,Coffee Shop,Diner
1,Alderwood / Long Branch,Pizza Place,Sandwich Place,Pub,Coffee Shop,Gym,Thai Restaurant,Fast Food Restaurant,Bakery,Bank,Bar
2,Allerton,Pizza Place,Grocery Store,Breakfast Spot,Pharmacy,Fast Food Restaurant,Thai Restaurant,Bakery,Bank,Bar,Café
3,Bathurst Manor / Wilson Heights / Downsview North,Bank,Coffee Shop,Diner,Sandwich Place,Restaurant,Pizza Place,Pharmacy,Ice Cream Shop,Sushi Restaurant,Thai Restaurant
4,Bayview Village,Bank,Café,Japanese Restaurant,Thai Restaurant,Gym,Bakery,Bar,Breakfast Spot,Clothing Store,Coffee Shop


<a id='item4'></a>

In [61]:
neighborhoods_venues_sorted.shape

(86, 11)

## Cluster Neighborhoods

Running *k*-means to cluster the neighborhood into 5 clusters.

In [138]:
# set number of clusters
kclusters = 9

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([5, 0, 0, 5, 5, 5, 5, 3, 5, 1, 2, 5, 5, 5, 5, 5, 1, 5, 5, 5, 1, 1,
       5, 1, 5, 4, 5, 5, 5, 2, 5, 5, 2, 1, 5, 5, 0, 1, 8, 5, 0, 5, 4, 5,
       5, 1, 4, 1, 2, 5, 2, 5, 7, 5, 5, 0, 4, 5, 5, 5, 4, 7, 5, 1, 1, 1,
       5, 5, 1, 5, 5, 5, 5, 6, 5, 1, 5, 5, 0, 0, 4, 1, 5, 8, 2, 4],
      dtype=int32)

In [139]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Agincourt,Breakfast Spot,Clothing Store,Thai Restaurant,Gym,Bakery,Bank,Bar,Café,Coffee Shop,Diner
1,0,Alderwood / Long Branch,Pizza Place,Sandwich Place,Pub,Coffee Shop,Gym,Thai Restaurant,Fast Food Restaurant,Bakery,Bank,Bar
2,0,Allerton,Pizza Place,Grocery Store,Breakfast Spot,Pharmacy,Fast Food Restaurant,Thai Restaurant,Bakery,Bank,Bar,Café
3,0,Bathurst Manor / Wilson Heights / Downsview North,Bank,Coffee Shop,Diner,Sandwich Place,Restaurant,Pizza Place,Pharmacy,Ice Cream Shop,Sushi Restaurant,Thai Restaurant
4,0,Bayview Village,Bank,Café,Japanese Restaurant,Thai Restaurant,Gym,Bakery,Bar,Breakfast Spot,Clothing Store,Coffee Shop


In [140]:
toronto_grouped_clustering.shape

(86, 25)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [141]:
if 'Cluster Labels' in neighborhoods_venues_sorted:
    #neighborhoods_venues_sorted['Cluster Labels'] == kmeans.labels_
    neighborhoods_venues_sorted.drop(columns=['Cluster Labels'],inplace = True)
    neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
else:
    neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
    

In [142]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,5,Agincourt,Breakfast Spot,Clothing Store,Thai Restaurant,Gym,Bakery,Bank,Bar,Café,Coffee Shop,Diner
1,0,Alderwood / Long Branch,Pizza Place,Sandwich Place,Pub,Coffee Shop,Gym,Thai Restaurant,Fast Food Restaurant,Bakery,Bank,Bar
2,0,Allerton,Pizza Place,Grocery Store,Breakfast Spot,Pharmacy,Fast Food Restaurant,Thai Restaurant,Bakery,Bank,Bar,Café
3,5,Bathurst Manor / Wilson Heights / Downsview North,Bank,Coffee Shop,Diner,Sandwich Place,Restaurant,Pizza Place,Pharmacy,Ice Cream Shop,Sushi Restaurant,Thai Restaurant
4,5,Bayview Village,Bank,Café,Japanese Restaurant,Thai Restaurant,Gym,Bakery,Bar,Breakfast Spot,Clothing Store,Coffee Shop


In [143]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


In [144]:
toronto_merged = Toronto_Neighb

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


In [145]:
toronto_merged.dropna(how='any', inplace = True)
toronto_merged["Cluster Labels"] = toronto_merged["Cluster Labels"].astype(int)

In [146]:
toronto_merged.groupby("Cluster Labels").count()

,Borough,Neighborhood,Latitude,Longitude,city,Postalcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Cluster Labels,,,,,,,,,,,,,,,,
0,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7
1,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18
2,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6
3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7
5,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47
6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
7,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
8,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2


In [147]:
toronto_merged.head()

,Borough,Neighborhood,Latitude,Longitude,city,Postalcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
298,Bronx,Allerton,40.865788,-73.859319,NY,NY,0,Pizza Place,Grocery Store,Breakfast Spot,Pharmacy,Fast Food Restaurant,Thai Restaurant,Bakery,Bank,Bar,Café
0,North York,Parkwoods,43.753259,-79.329656,TO,M3A,4,Park,Thai Restaurant,Gym,Bakery,Bank,Bar,Breakfast Spot,Café,Clothing Store,Coffee Shop
1,North York,Victoria Village,43.725882,-79.315572,TO,M4A,0,Pizza Place,Coffee Shop,Thai Restaurant,Gym,Bakery,Bank,Bar,Breakfast Spot,Café,Clothing Store
2,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,TO,M5A,5,Coffee Shop,Bakery,Pub,Park,Restaurant,Breakfast Spot,Café,Bank,Ice Cream Shop,Thai Restaurant
3,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763,TO,M6A,5,Clothing Store,Coffee Shop,Thai Restaurant,Gym,Bakery,Bank,Bar,Breakfast Spot,Café,Diner


In [148]:
#Get the NY Cluster
toronto_merged[toronto_merged.city=="NY"]

,Borough,Neighborhood,Latitude,Longitude,city,Postalcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
298,Bronx,Allerton,40.865788,-73.859319,NY,NY,0,Pizza Place,Grocery Store,Breakfast Spot,Pharmacy,Fast Food Restaurant,Thai Restaurant,Bakery,Bank,Bar,Café


In [149]:
NY_cluster = toronto_merged[toronto_merged.city=="NY"].iloc[0,toronto_merged.columns.tolist().index('Cluster Labels')]
NY_cluster

0

#### Cluster for the Accidental Tourist: you may now choose the Toronto Neighborhood that is most similiar with your NY Neighborhood, and book your ticket and reservations :)

In [150]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == NY_cluster, toronto_merged.columns[[1] + [2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Latitude,Postalcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
298,Allerton,40.865788,NY,0,Pizza Place,Grocery Store,Breakfast Spot,Pharmacy,Fast Food Restaurant,Thai Restaurant,Bakery,Bank,Bar,Café
1,Victoria Village,43.725882,M4A,0,Pizza Place,Coffee Shop,Thai Restaurant,Gym,Bakery,Bank,Bar,Breakfast Spot,Café,Clothing Store
8,Parkview Hill / Woodbine Gardens,43.706397,M4B,0,Pizza Place,Bank,Pharmacy,Fast Food Restaurant,Thai Restaurant,Grocery Store,Bakery,Bar,Breakfast Spot,Café
50,Humber Summit,43.756303,M9L,0,Pizza Place,Thai Restaurant,Gym,Bakery,Bank,Bar,Breakfast Spot,Café,Clothing Store,Coffee Shop
70,Westmount,43.696319,M9P,0,Sandwich Place,Pizza Place,Coffee Shop,Thai Restaurant,Grocery Store,Bakery,Bank,Bar,Breakfast Spot,Café
77,Kingsview Village / St. Phillips / Martin Grov...,43.688905,M9R,0,Sandwich Place,Pizza Place,Thai Restaurant,Grocery Store,Bakery,Bank,Bar,Breakfast Spot,Café,Clothing Store
93,Alderwood / Long Branch,43.602414,M8W,0,Pizza Place,Sandwich Place,Pub,Coffee Shop,Gym,Thai Restaurant,Fast Food Restaurant,Bakery,Bank,Bar


In [151]:
address = 'Toronto, TO'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [152]:
# create map
map_cluster_NY = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    if cluster == NY_cluster:    
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster-1],
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7).add_to(map_cluster_NY)
    
map_cluster_NY